### I really enjoy dunking. It mixes talent, technique, and skill as well as creating interesting challenges in training. Naturally I tend to get along with others who enjoy spending time in the same niche as me.
### But where in the world are other dunkers located?

#### Name and location data pulled from wda.do 

# IMPORTS

In [1]:
# Imports
import folium
import folium.plugins as plugins
from geopy.geocoders import Nominatim
from pygeocoder import Geocoder
#--------------------------------------
import requests
from bs4 import BeautifulSoup
import pandas as pd
import ast
#--------------------------------------
import sys
from config import API_KEY
print("Finished imports")

Finished imports


In [3]:
#  Different types of levels of dunkers.
level_list = ["nba", "nba_retired", "pro", "amateur", "retired"]
#  Create a list of URLS to scrape location and name information from
url_list = [f"https://wda.do/dunkers?kind={level}" for level in level_list]

# Opens Dunkers txt file and updates dunker names and locations
with open("Dunkers.txt", "w") as f:
    dunker_sum = 0
    name_list = []
    location_list = []
    league_list = []
    f.write("name-location")
    for url in url_list:
        response = requests.get(url).text
        soup = BeautifulSoup(response, "html.parser")
        dunker_name_list = soup.find_all("h1", class_="h1")
        dunker_location_list = soup.find_all("div", class_="country-block")
        league = soup.find_all("div", class_="dunker-kind")
        for l in league:
            league_list.append("AMATEUR" if l.text == "\n" else l.text.strip())
        for i, name in enumerate(dunker_name_list):
            location_name = dunker_location_list[i].text.strip()
            location_list.append(location_name)
            name_list.append(name.text)

            try:
                # Strip preceding and trailing whitespace from location text
                location_name = dunker_location_list[i].text.strip()

                f.write(f"-{name.text}-{location_name}")
            except UnicodeEncodeError:
                f.write(f"-{name.text}-N/A")
            dunker_sum += 1

    name_and_location = zip(name_list, location_list)
    dunkerdf = pd.DataFrame(list(zip(name_list, location_list)), columns=["Name", "Location"])

print(f"Done writing {dunker_sum} dunkers to file")



Done writing 175 dunkers to file


In [2]:
coord_dict = {"United States":(37.0902, -95.7129)}  # Dictionary to hold coordinates for common locations

def get_coordinates(loc_list):
    # Converts location to geocode 
    # Retrieves geocode for location if location in coord_dict
    # Returns list of all geocode/coordinates
    coordinate_list = []
    geo = Geocoder(api_key=API_KEY)
    for location in loc_list:
        if coord_dict.get(location) is None:
            result = geo.geocode(location)
            coords = result.coordinates
            coordinate_list.append(coords)
        else:
            coordinate_list.append(coord_dict.get(location))
    return coordinate_list  

# vvv Only run to update Dunker DataFrame vvv

In [27]:
# Add coordinates column to DataFrame
dunkerdf["Coordinates"] = get_coordinates(dunkerdf["Location"])
dunkerdf["League"] = league_list
dunkerdf.head()
dunkerdf.to_csv("Dunker_data", index=False)

### Let's read in the csv and see what it looks like

In [3]:
df = pd.read_csv("Dunker_data")
df.head()

,Name,Location,Coordinates,League
0,DeMar DeRozan,United States,"(37.0902, -95.7129)",NBA
1,Hamidou Diallo,United States,"(37.0902, -95.7129)",NBA
2,Jeremy Evans,United States,"(37.0902, -95.7129)",NBA
3,Aaron Gordon,United States,"(37.0902, -95.7129)",NBA
4,Gerald Green,United States,"(37.0902, -95.7129)",NBA


In [4]:
m = folium.Map()  # Instantiate folium Map object

In [5]:
cluster_coord_list = []
name_list = []
total_name = []
# Lists to hold location and popup information for Markers

# If there are dunkers in the DataFrame that share location coordinates, they must be added to a MarkerCluster.
# Otherwise a regular Marker will do.
# league_by_color = df.loc[df["Coordinates"] == coordin]["League"].item()

value_series = df.Coordinates.value_counts()
unique_idx_list = value_series.index
for coordin in unique_idx_list:
    
    if value_series[coordin] > 1:
        new_df = df.loc[df.Coordinates==coordin]
        total_name = [name for name in new_df["Name"]]
        total_coord = [coord for coord in new_df["Coordinates"]]
        new = list(map(ast.literal_eval, total_coord))
        plugins.MarkerCluster(locations=new, popups=total_name).add_to(m)
        
    else:
        folium.Marker(location=ast.literal_eval(coordin), popup=df.loc[df["Coordinates"] == coordin]["Name"].item(), icon=folium.Icon(color="black")).add_to(m)


In [6]:
# Display the map
m